In [9]:
from pylabnet.hardware.awg.quantum_machine import Driver
from pylabnet.utils.logging.logger import LogClient
from configuration import config

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


In [3]:
dev_id = 'QM_ford'

# Instantiate
logger = LogClient(
    host='192.168.50.101', 
    port=38967,
    module_tag=f'Quantum Machine {dev_id}'
)

In [5]:
# Instanciate Hardware class
qm = Driver(dev_id, logger=logger)

2022-02-26 17:13:32,070 - qm - INFO - Performing health check
2022-02-26 17:13:32,074 - qm - INFO - Health check passed


In [11]:
from qm.qua import program, infinite_loop_, play, wait

# programe
print('start...')
with program() as prog:
    with infinite_loop_():
        play('trigger', 'tt') 
        # play('X', 'e_spin')
        wait(100, 'tt')

qm.execute(prog, config)

start...
2022-02-26 17:52:03,631 - qm - INFO - Flags: 
2022-02-26 17:52:03,632 - qm - INFO - Sending program to QOP
2022-02-26 17:52:03,664 - qm - INFO - Executing program
